Analise de Sentimento realtime no Twitter

** Configurações utilizadas **

Vamos utilizar Framework Spark para processar dados do Twitter e utilizar API do próprio twitter para captura dos dados

In [0]:
!pip install tweepy
!pip install textblob

     |████████████████████████████████| 101 kB 2.6 MB/s 
     |████████████████████████████████| 151 kB 69.4 MB/s 
     |████████████████████████████████| 62 kB 52 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.26.0
    Not uninstalling requests at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-f480f542-9689-4944-8a88-2bc40672a314
    Can't uninstall 'requests'. No files were found to uninstall.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-f480f542-9689-4944-8a88-2bc40672a314/bin/python -m pip install --upgrade pip' command.


In [0]:
# Modulos usados
import numpy as np
import pandas as pd
import tweepy as tw
import matplotlib as plt
import seaborn as sns
import textblob
from IPython.display import display
from textblob import TextBlob
import re
from time import gmtime, strftime
import time


In [0]:
# Autenticação do Twitter, necessario gerar a alteticação no modo conta de desenvolvedor
#colocar dentro das aspas simples os seus dados de acesso a API do Twitter

bearer_token = 'xxxx'
consumer_key = 'xxxx'
consumer_secret = 'xxxx'
access_token = 'xxxx'
access_token_secret = 'xxxx'


In [0]:
cliente = tw.Client(bearer_token=bearer_token, consumer_key=consumer_key, consumer_secret=consumer_secret, access_token=access_token, access_token_secret=access_token_secret)

In [0]:
#Odate Batch de pesquisa

#start = '2023-02-10T12:30:01Z'
#end = '2023-02-15T20:50:01Z'

#  Termo de busca (Twitter abre uma porta para os desenvolvedores)
search_tweet = cliente.search_recent_tweets(query='#SANB11',max_results=100)

In [0]:
search_tweet

Out[556]: Response(data=[<Tweet id=1625573282496782336 text='#MERCADOS: Santander cai com troca de posições no setor\n\nAs ações do Santander (#SANB11) registram a maior baixa do setor bancário no Ibovespa, com perda de 2,11% (R$ 28,27). Analistas dizem que os investidores estão migrando para os papéis do Banco do B… https://t.co/e1pfNuy1hA'>, <Tweet id=1625543916727549988 text='Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 e 2023 e comparação com o resultado reportado. https://t.co/EwbAcKAZCN'>, <Tweet id=1625520420244815873 text='RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 e…'>, <Tweet id=1625511604652544000 text='RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 e…'>, <Tweet id=1625498703632138250 text='RT @candremv: Comp

In [0]:
# Imprimindo Tweets
tweets = search_tweet.data
base = []

for i in tweets:        
    texto = i.text
    
    if('RT' in texto):
        posicao = texto.find(':')
        texto = texto[posicao+2:]
        dados = texto.lower()
    base.append(dados)

# Agrupando os tweets em um dataframe
df = pd.DataFrame(base,columns=['texto_twitter'])
pd.options.display.max_colwidth = 130 
df.head()

,texto_twitter
0,o santander #sanb11 provisionou ~30% de sua exposição à americanas.\n\no itaú unibanco #itub4 provisionou 100% de sua exposição…
1,o santander #sanb11 provisionou ~30% de sua exposição à americanas.\n\no itaú unibanco #itub4 provisionou 100% de sua exposição…
2,comparativo de resultados dos 4 grandes bancos listados #bbas3 #bbdc4 #itub4 #sanb11\n\nguidance do banco do brasil para 2022 e…
3,comparativo de resultados dos 4 grandes bancos listados #bbas3 #bbdc4 #itub4 #sanb11\n\nguidance do banco do brasil para 2022 e…
4,comparativo de resultados dos 4 grandes bancos listados #bbas3 #bbdc4 #itub4 #sanb11\n\nguidance do banco do brasil para 2022 e…


In [0]:
data = df
data['texto_twitter'] = np.array([tweet.text for tweet in tweets])
#Retirando os duplicados
#df.drop_duplicates(['texto_twitter'], inplace=True)

In [0]:
data

,texto_twitter
0,#MERCADOS: Santander cai com troca de posições no setor\n\nAs ações do Santander (#SANB11) registram a maior baixa do setor ba...
1,Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 ...
2,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...
3,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...
4,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...
5,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...
6,Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 ...
7,"RT @InvestidoresO: O Santander (SANB11) conseguiu na Justiça de São Paulo, a abertura do sigilo de e-mails de Beto Sicupira, u..."
8,@joaocontador11 A mudança na cobertura t/t não foi tão significativa (-8pp). Se considerássemos o caso Americanas 100% provisi...
9,"🚨\nPessoal, live AGORA com Max Bohm para TODOS.\nAberta para perguntas de papéis, macro, setores, etc.\n\nhttps://t.co/10VTbJO..."


In [0]:
# def clean_tweet(df):
#        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|       (\w+:\/\/\S+)", " ", df).split())


In [0]:
def analize_sentiment(df):
    analysis = TextBlob(clean_tweet(df))
    if analysis.sentiment.polarity > 0:
       return 1
    elif analysis.sentiment.polarity == 0:
       return 0
    else:
       return -1

In [0]:
data['SA'] = np.array([ analize_sentiment(tweet) for tweet in data['texto_twitter'] ])

In [0]:
data

,texto_twitter,SA
0,#MERCADOS: Santander cai com troca de posições no setor\n\nAs ações do Santander (#SANB11) registram a maior baixa do setor ba...,0
1,Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 ...,0
2,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
3,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
4,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
5,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
6,Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 ...,0
7,"RT @InvestidoresO: O Santander (SANB11) conseguiu na Justiça de São Paulo, a abertura do sigilo de e-mails de Beto Sicupira, u...",0
8,@joaocontador11 A mudança na cobertura t/t não foi tão significativa (-8pp). Se considerássemos o caso Americanas 100% provisi...,0
9,"🚨\nPessoal, live AGORA com Max Bohm para TODOS.\nAberta para perguntas de papéis, macro, setores, etc.\n\nhttps://t.co/10VTbJO...",1


In [0]:
pos_tweets = [ tweet for index, tweet in enumerate(data['texto_twitter']) if data['SA'][index] > 0]
neu_tweets = [ tweet for index, tweet in enumerate(data['texto_twitter']) if data['SA'][index] == 0]
neg_tweets = [ tweet for index, tweet in enumerate(data['texto_twitter']) if data['SA'][index] < 0]
# Printando as porcentagens:
print("Percentage of positive tweets: {}%".format(len(pos_tweets)*100/len(data['texto_twitter'])))
print("Percentage of neutral tweets: {}%".format(len(neu_tweets)*100/len(data['texto_twitter'])))
print("Percentage de negative tweets: {}%".format(len(neg_tweets)*100/len(data['texto_twitter'])))

Percentage of positive tweets: 4.166666666666667%
Percentage of neutral tweets: 95.83333333333333%
Percentage de negative tweets: 0.0%


In [0]:
# Grava os resultados
#rdd_save = '/FileStore/tables/datatraining/r'+time.strftime("%I,%M%S")
resultados_rdd = df
df.to_parquet('twitter-bbb.parquet')

#df.to_csv("your_name.csv", columns = ['texto_twitter', 'SA'])

dbutils.fs.put("/FileStore/tables/datatraining/twitter-bbb.parquet", resultados_rdd)

#resultados_rdd.write.format("csv").save("/FileStore/tables/datatraining/twitter-bbb2.csv")
#dbutils.fs.put("/FileStore/my-stuff/my-file.txt", "This is the actual text that will be saved to disk. Like a 'Hello world!' example")

#resultados_rdd.saveAsTextFile(rdd_save)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-1140900483863026> in <cell line: 8>()
      6 #df.to_csv("your_name.csv", columns = ['texto_twitter', 'SA'])
      7 
----> 8 dbutils.fs.put("/FileStore/tables/datatraining/twitter-bbb.parquet", resultados_rdd)
      9 
     10 #resultados_rdd.write.format("csv").save("/FileStore/tables/datatraining/twitter-bbb2.csv")

/databricks/python_shell/dbruntime/dbutils.py in f_with_exception_handling(*args, **kwargs)
    344             def f_with_exception_handling(*args, **kwargs):
    345                 try:
--> 346                     return f(*args, **kwargs)
    347                 except Py4JJavaError as e:
    348 

/databricks/python_shell/dbruntime/dbutils.py in put(self, file, contents, overwrite)
    403         @prettify_exception_message
    404         def put(self, file, contents, overwrite=False):
--> 405             

In [0]:
df

,texto_twitter,SA
0,#MERCADOS: Santander cai com troca de posições no setor\n\nAs ações do Santander (#SANB11) registram a maior baixa do setor ba...,0
1,Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 ...,0
2,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
3,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
4,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
5,RT @candremv: Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Bra...,0
6,Comparativo de resultados dos 4 grandes bancos listados #BBAS3 #BBDC4 #ITUB4 #SANB11\n\nGuidance do Banco do Brasil para 2022 ...,0
7,"RT @InvestidoresO: O Santander (SANB11) conseguiu na Justiça de São Paulo, a abertura do sigilo de e-mails de Beto Sicupira, u...",0
8,@joaocontador11 A mudança na cobertura t/t não foi tão significativa (-8pp). Se considerássemos o caso Americanas 100% provisi...,0
9,"🚨\nPessoal, live AGORA com Max Bohm para TODOS.\nAberta para perguntas de papéis, macro, setores, etc.\n\nhttps://t.co/10VTbJO...",1
